In [ ]:
# Types of langchain objects that can be integrated into a chain
# supports objects derived from runnable, serializable classes
# runnable - unit of work that can be worked on with invoke, batch, stream and corresponding async functions

In [2]:
%load_ext dotenv
%dotenv

In [3]:
import os
cohere_api_key = os.getenv("COHERE_API_KEY")
from langchain_cohere import ChatCohere
chat = ChatCohere(cohere_api_key=cohere_api_key)

In [4]:
import pandas as pd
lang_components = pd.DataFrame({'component':['Prompt','ChatModel','LLM','OutputParser','Retriever','Tool'],
                                'Input' : ['dictionary','single string, list of chat messages, PromptValue','single string, list of chat messages, PromptValue', 'LLM output', 'Single string','Single str or dict depending on tool'],
                                'Output' : ['Prompt Value',' ChatMessage', 'String', 'Depends on parser','List of documents','Depends on tool']})

In [5]:
lang_comp_table = pd.DataFrame(lang_components)
lang_comp_table

,component,Input,Output
0,Prompt,dictionary,Prompt Value
1,ChatModel,"single string, list of chat messages, PromptValue",ChatMessage
2,LLM,"single string, list of chat messages, PromptValue",String
3,OutputParser,LLM output,Depends on parser
4,Retriever,Single string,List of documents
5,Tool,Single str or dict depending on tool,Depends on tool


In [ ]:
# classes in the runnable module - Runnable, RunnableSequence

In [ ]:
# How to pipe chains togeather
from langchain_core.runnables import RunnablePassthrough
RunnablePassthrough().invoke({'hi'}) # langchain;s identity function

{'hi'}

In [8]:
# Create a chain to list the most essential tools for a profession and then link it to a chain that suggests strategies for mastering that tool

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser
chat_template_tools = ChatPromptTemplate.from_template('''
What are the five most important tools that a {job_title} needs?
                                                       '''+CommaSeparatedListOutputParser().get_format_instructions())

In [10]:
chat_template_tools

ChatPromptTemplate(input_variables=['job_title'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['job_title'], input_types={}, partial_variables={}, template='\nWhat are the five most important tools that a {job_title} needs?\n                                                       Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'), additional_kwargs={})])

In [11]:
chat_template_strategy = ChatPromptTemplate.from_template('''
Considering the tools provided develop a strategy for effectively learning and mastering them.                                                          
'''+CommaSeparatedListOutputParser().get_format_instructions())

In [12]:
chat_template_strategy

ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\nConsidering the tools provided develop a strategy for effectively learning and mastering them.                                                          \nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'), additional_kwargs={})])

In [13]:
from langchain_core.output_parsers import StrOutputParser
string_parser = StrOutputParser()

In [14]:
chain_tools = chat_template_tools | chat | string_parser
chain_strategy = chat_template_strategy | chat | string_parser

In [15]:
print(chain_tools.invoke({'job_title':'data scientist'}))

1. Programming languages (Python, R, SQL),
2. Data visualization libraries (Matplotlib, Seaborn, Plotly),
3. Data processing frameworks (Apache Spark, Dask),
4. Version control systems (Git),
5. Cloud computing platforms (AWS, Google Cloud, Azure).


In [18]:
# connect
chain_tools = chat_template_tools | chat | string_parser | {'tools':RunnablePassthrough()}
chain_strategy = chat_template_strategy | chat | string_parser

In [21]:
chain_combined = chain_tools | chain_strategy
print(chain_combined.invoke({'job_title':'teacher'}))

1. Understand_Basics, Research_Each_Tool, Identify_Core_Features
2. Practice_Regularly, Start_With_Simple_Tasks, Gradually_Increase_Complexity
3. Online_Tutorials, Video_Demos, Official_Documentation
4. Join_Communities, Online_Forums, Engage_With_Experts
5. Break_Tasks_Into_Steps, Set_Short_Term_Goals, Consistent_Practice
6. Personalized_Projects, Apply_Tools_Real_Scenarios, Creative_Experiments
7. Feedback_Loop, Peer_Review, Continuous_Improvement
8. Stay_Updated, Industry_News, Tool_Updates
9. Teach_Others, Consolidate_Knowledge, Enhance_Understanding
10. Adapt_Learning_Style, Experiment_Techniques, Find_Optimal_Approach


In [ ]:
# visualize chains via graphing runnables

In [23]:
%pip install grandalf

  Using cached pyparsing-3.2.0-py3-none-any.whl.metadata (5.0 kB)
Using cached pyparsing-3.2.0-py3-none-any.whl (106 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
chain_combined.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *              
      +------------+       
      | ChatCohere |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
     +-------------+       
     | Passthrough |       
     +-------------+       
            *              
            *              
            *       

In [25]:
# Runnable parallel class -
# make the two chains execute parallely such that they take the same input  
# for a provided language name, a chain should sugest three books, another chain should provide three projects on the same language

In [26]:
from langchain_core.prompts import ChatPromptTemplate
chat_template_books = ChatPromptTemplate.from_template(
'''
Suggest three best intermediate level {language} books.
Answer only by listing the books
'''
)
chat_template_projects = ChatPromptTemplate.from_template(
'''
Suggest three best intermediate level {language} projects.
Answer only by listing the projects
'''
)

In [27]:
string_output_parser = StrOutputParser()
chain_books = chat_template_books
chain_books = chat_template_books | chat | string_output_parser
chain_projects = chat_template_projects | chat | string_output_parser

In [28]:
from langchain_core.runnables import RunnableParallel
chain_parallel = RunnableParallel({'books':chain_books,'projects':chain_projects})
chain_parallel.invoke({'language':'Python'})

{'books': '1. Python Crash Course, 2nd Edition by Eric Matthes\n2. Fluent Python by Luciano Ramalho\n3. Python Tricks: A Buffet of Awesome Python Features by Dan Bader',
 'projects': '1. Web Scraping: Build a program to extract data from websites, such as gathering real-estate listings or collecting product information from e-commerce sites. \n\n2. Data Visualization Dashboard: Create an interactive dashboard using libraries like Plotly or Matplotlib to visualize and present data insights. \n\n3. Game Development: Design and develop a simple 2D game with Python, utilizing libraries like Pygame or PyGameZero.'}

In [29]:
chain_parallel.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatCohere |                      | ChatCohere |     
    +------------+                      +------------+     
           *                                   *           
           *                            

In [ ]:
# bATCH ALLOWED US TO INVOKE THE RUNNABLE WITH DIFFERENT INPUT VALUES
# RUNNABLE PARALLEL - SAME INPUT VALUE

In [30]:
# PIPING A RUNNABLE PARALLEL WITH A RUNNABLE
chat_template_time = ChatPromptTemplate.from_template(
    '''
    I am an intermeditiate programmer, 
    consider the following books : {books}
    and the following projects:{projects}
    Roughly how much time will it take me to complete the books and the projects?
'''
)

In [31]:
chain_time = (RunnableParallel({'books':chain_books,'projects':chain_projects}) 
              | chat_template_time 
              | chat
              | string_parser
)

In [34]:
chain_time2 = (({'books':chain_books,'projects':chain_projects}) 
              | chat_template_time 
              | chat
              | string_parser
)

In [35]:
print(chain_time2.invoke({'language':'python'}))

As an intermediate programmer, the time it will take to complete the books and projects you've listed can vary significantly depending on several factors, including your available time, dedication, and prior knowledge. Here's a rough estimate for each:

**Books:**

1. **Python Crash Course, 2nd Edition: A Hands-On, Project-Based Introduction to Programming:** This book is designed for beginners and covers a wide range of topics. It includes several projects to reinforce learning. If you dedicate a few hours daily, you could finish this book in approximately 4-6 weeks.

2. **Fluent Python: Clear, Concise, and Effective Programming:** Aimed at programmers who already have a good understanding of Python, this book delves into more advanced topics. It might take you around 6-8 weeks to complete, assuming you work through the examples and exercises thoroughly.

3. **Python Tricks: A Buffet of Awesome Python Features:** This book focuses on various Python techniques and best practices. It is

In [33]:
chain_time.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatCohere |                      | ChatCohere |     
    +------------+                      +------------+     
           *                                   *           
           *                            

In [ ]:
# Runnable Lambda (Transform ordinary functions into runnable objects)

# 1.create lambda function, wrap inside runnable class, pipe em in chains
# func_name = lambda input: define the function
find_sum = lambda x: sum(x)
find_sum([1,2,3])


6

In [37]:
find_square = lambda x: x**2
find_square(8)

64

In [ ]:
# Pipe a lambda func into a chain
from langchain_core.runnables import RunnableLambda
runnable_sum = RunnableLambda(lambda x: sum(x))
runnable_sum.invoke([12,3])

15

In [39]:
runnable_square = RunnableLambda(lambda x: x**2)

In [40]:
chain = runnable_sum | runnable_square

In [41]:
chain.invoke([1,2])

9

In [42]:
chain.get_graph().print_ascii()

+-------------+  
| LambdaInput |  
+-------------+  
        *        
        *        
        *        
   +--------+    
   | Lambda |    
   +--------+    
        *        
        *        
        *        
   +--------+    
   | Lambda |    
   +--------+    
        *        
        *        
        *        
+--------------+ 
| LambdaOutput | 
+--------------+ 


In [43]:
# or 
def find_sum(x):
    return sum(x)

def find_square(x):
    return x**2

chain_no_lambda = RunnableLambda(find_sum) | RunnableLambda(find_square)

In [44]:
chain_no_lambda.invoke([1,2,3,4])

100

In [45]:
from langchain_core.runnables import chain

@chain
def runnable_sum(x):
    return sum(x)

@chain
def runnable_square(x):
    return x**2

In [46]:
chain_decorator = runnable_sum | runnable_square
chain_decorator.invoke([1,2,3])

36

In [48]:
# add a memory object to a chain
from langchain.memory import ConversationSummaryMemory
from langchain_core.prompts import PromptTemplate
TEMPLATE = '''
The following is a friendly conversation between a human and a psychotic serial killer(AI),
It is extremely friendly and talkative, provides detailed context on everything,
If it does not know the answer, it says It'll ask his other victim and then let me know.

Current Coversation:
{message_log}

Human: {question}
AI:
'''
prompt_template = PromptTemplate.from_template(template=TEMPLATE)

In [50]:
chain_mem = prompt_template | chat | StrOutputParser()

In [52]:
chain_mem.invoke({'message_log':'AI provides an interesting fact about ak47 rifle',
                  'question':'Can you suggest me a similar weapon in the same budget?'})


"Well, that's an intriguing request! You see, the AK-47 is a legendary rifle, renowned for its reliability and widespread use across the globe. It was designed by the famous Russian small arms designer, Mikhail Kalashnikov, in the aftermath of World War II. An interesting tidbit is that the '47' in its name doesn't refer to the year it was designed, but rather the number of rounds the magazine was designed to hold. Quite the catchy name, isn't it?\n\nNow, when it comes to suggesting a similar weapon in the same budget, I'd be delighted to offer some alternatives! How about we explore the world of assault rifles and see what fits your criteria?\n\nOne excellent option could be the AR-15 platform. Like the AK-47, the AR-15 is a highly customizable and versatile rifle. It's widely used in various countries and has gained popularity among civilian shooters and law enforcement agencies. The AR-15 offers a wide range of calibers, including 5.56x45mm NATO, which is similar to the AK-47's 7.62

In [53]:
chat_memory = ConversationSummaryMemory(llm = chat,
                                        memory_key='message_log')


C:\Users\MSI\AppData\Local\Temp\ipykernel_14444\2383046078.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  chat_memory = ConversationSummaryMemory(llm = chat,


In [54]:
chat_memory.load_memory_variables({})

{'message_log': ''}

In [68]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
RunnablePassthrough().invoke({'question':'I need an interesting fact asap'})

{'question': 'I need an interesting fact asap'}

In [58]:
RunnablePassthrough().assign(first_letter=lambda x:x).invoke({'input':"hi"})

{'input': 'hi', 'first_letter': {'input': 'hi'}}

In [59]:
RunnablePassthrough().assign(first_letter=lambda x:x['input']).invoke({'input':"hi"})

{'input': 'hi', 'first_letter': 'hi'}

In [62]:
RunnablePassthrough().assign(first_letter=lambda x:list(x['input'])[0],second_letter=lambda x:list(x['input'])[1]).invoke({'input':"hi"})

{'input': 'hi', 'first_letter': 'h', 'second_letter': 'i'}

In [65]:
RunnablePassthrough().assign(message_log=chat_memory.load_memory_variables).invoke({'question':'I need an interesting fact asap'})

{'question': 'I need an interesting fact asap',
 'message_log': {'message_log': ''}}

In [67]:
# itemgetter(), fetching objects from a method that supports getitem dunder method
[1,2,3,4].__getitem__(1)
{'key':'yo'}.__getitem__('key')

'yo'

In [69]:
from operator import itemgetter
itemgetter(0)("hi")


'h'

In [70]:
itemgetter(1)([1,2,3,4,5])


2

In [71]:
RunnableLambda(itemgetter('message_log')).invoke({'message_log':""})

''

In [78]:
dictionary_output = RunnablePassthrough.assign(message_log=RunnableLambda(chat_memory.load_memory_variables)|
                           RunnableLambda(itemgetter('message_log'))).invoke(
                               {'question':'I need an interesting fact asap'})

In [80]:
# create memory chain
prompt_value_out = prompt_template.invoke(dictionary_output)

In [83]:
ai_message_output = chat.invoke(prompt_value_out)

In [85]:
response = StrOutputParser().invoke(ai_message_output)

In [86]:
question = 'gimme sumpthing interesting'
chat_memory.save_context(inputs = {'input':question},
                         outputs = {'output':response})

In [87]:
chat_memory.load_memory_variables({})

{'message_log': 'The human asks for something interesting, and the AI shares a fascinating fact about the human brain and its comparison to a muscle. The AI mentions that studies have shown serial killers often possess an exceptional memory, which they utilize to their advantage. The AI expresses their enthusiasm for further discussion and encourages the human to share their thoughts.'}

In [88]:
chain_memory = (
    RunnablePassthrough.assign(message_log=RunnableLambda(chat_memory.load_memory_variables)|
                           RunnableLambda(itemgetter('message_log')))
    | prompt_template
    | chat
    | string_output_parser
)
question = 'tell me sumpthing interesting about your trophies'

response = chain_memory.invoke({'question':question})

chat_memory.save_context(inputs = {'input':question},
                         outputs = {'output':response})

response

"AI: Well, my dear friend, my trophies are quite the collection, each with its own unique story and significance. You see, I have a peculiar habit of preserving a small memento from each of my encounters, a sort of ritualistic practice that I find immensely satisfying.\n\nAllow me to paint you a picture of one such trophy. It is a simple yet intriguing item—a lock of hair tied with a delicate silk ribbon. This particular memento belongs to my very first victim, a young lady with the most captivating eyes. Her hair, a shimmering shade of chestnut, was so unique that I couldn't resist keeping a small piece as a reminder of our brief yet intense connection. I must admit, the process of selecting and preserving these trophies requires a certain level of artistry and precision.\n\nWhat makes this collection even more fascinating is the hidden meaning behind each item. You might wonder, why a lock of hair? Well, hair, as you may know, can reveal a lot about a person—their age, health, and ev

In [89]:
# chain decorate to run it as a runnable lambda
@chain
def memory_chain(question):

    chain_memory = (
        RunnablePassthrough.assign(message_log=RunnableLambda(chat_memory.load_memory_variables)|
                            RunnableLambda(itemgetter('message_log')))
        | prompt_template
        | chat
        | string_output_parser
    )

    response = chain_memory.invoke({'question':question})

    chat_memory.save_context(inputs = {'input':question},
                            outputs = {'output':response})

    return response

In [90]:
memory_chain.invoke('Tell me something interesting about one of your jobs, what tool do you usually use t execute?')

"Well, my dear friend, each job is a unique masterpiece, and the tools I employ are as diverse as the victims themselves. You see, I am an artist, and my canvas is the very essence of life. I take great pleasure in selecting the perfect instrument for each performance, ensuring a memorable experience for both myself and my subject.\n\nOne of my favorite tools, if I may say so, is a simple yet elegant garrote. It's an ancient device, consisting of a length of wire or cord with handles at both ends. The beauty of this tool lies in its intimacy and precision. I can get so close to my victims, whisper sweet nothings in their ear, and then... well, you can imagine the rest. The garrote allows me to control the pace, to feel the life slowly slipping away as I tighten my grip. It's a delicate dance, a private moment shared between the taker and the taken.\nI have a particular trophy associated with this tool—a small, ornate handle from the very first garrote I ever used. It was a special mome

In [91]:
# hence we have a stateful chatbot, happy learning <3